# Define StateSchema

In [1]:
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing_extensions import Annotated, Sequence, TypedDict

class MessagesState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

## Create a new Chat

In [2]:
from utils import ChatHistoryManager
chat = ChatHistoryManager.new_chat()
print(f"Chat ID:", chat.chat_id)

Chat ID: a10b561e-d2aa-4b08-9d78-441ceeba8f0f


In [3]:
from utils import (
    YTContentData,
    webvtt_2_str,
    download_youtube_audio_or_transcript,
)

yt_video_data: YTContentData = download_youtube_audio_or_transcript(
    video_url="https://www.youtube.com/watch?v=W3I3kAg2J7w&t=231s"
)

vtt_str = webvtt_2_str(vtt_file_path=yt_video_data.transcript_path)

print(vtt_str[:100])    # print first 100 characters

2025-07-07 10:56:20.873 | DEBUG    | utils.youtube:download_youtube_audio_or_transcript:133 - Cache exists, validating cache...
2025-07-07 10:56:20.874 | DEBUG    | utils.youtube:download_youtube_audio_or_transcript:145 - Cache contains the requested data. Using cache.


Last year, Grant gave the math department's Michael E. Moody
lecture on where math and physics colli


## Implement RAG

In [4]:
# ! pip install -U langchain-qdrant
import os
from rag import TextDocumentRAG
from langchain_core.documents import Document

document_rag = TextDocumentRAG(
    path_on_disk=chat.chat_dirpath,
    collection_path_on_disk=os.path.join(chat.chat_dirpath, "collection"),
    collection_name=chat.chat_id,
)

# add youtube transcripts to the vector database
document_rag.add_documents(
    [Document(vtt_str)], split_text=True,
    split_config={
        "chunk_size": 512,
        "chunk_overlap": 128
    },
    skip_if_collection_exists=True,
)

ValidationError: 1 validation error for HuggingFaceEmbeddings
model
  Extra inputs are not permitted [type=extra_forbidden, input_value='sentence-transformers/all-MiniLM-L12-v2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden